# Import libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [2]:
Body_Acc_X_train = pd.read_csv("/content/body_acc_x_train.txt", sep='\s+', header=None)
Body_Acc_y_train = pd.read_csv("/content/body_acc_y_train.txt", sep='\s+', header=None)
Body_Acc_z_train = pd.read_csv("/content/body_acc_z_train.txt", sep='\s+', header=None)
Body_Gyro_X_train = pd.read_csv("/content/body_gyro_x_train.txt", sep='\s+', header=None)
Body_Gyro_y_train = pd.read_csv("/content/body_gyro_y_train.txt", sep='\s+', header=None)
Body_Gyro_z_train = pd.read_csv("/content/body_gyro_z_train.txt", sep='\s+', header=None)
Total_Acc_X_train = pd.read_csv("/content/total_acc_x_train.txt", sep='\s+', header=None)
Total_Acc_y_train = pd.read_csv("/content/total_acc_y_train.txt", sep='\s+', header=None)
Total_Acc_z_train = pd.read_csv("/content/total_acc_z_train.txt", sep='\s+', header=None)


train_data = pd.concat([Body_Acc_X_train, Body_Acc_y_train, Body_Acc_z_train, Body_Gyro_X_train, Body_Gyro_y_train, Body_Gyro_z_train, Total_Acc_X_train, Total_Acc_y_train, Total_Acc_z_train], axis=1)

train_data.to_csv("train_data.csv", index=False)

In [3]:
train_data.shape

(7352, 1152)

In [4]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.000181,0.010139,0.009276,0.005066,0.010810,0.004045,0.004757,0.006214,0.003307,0.007572,...,0.100058,0.098564,0.093177,0.088742,0.090505,0.094843,0.098350,0.100385,0.099874,0.094987
1,0.001094,0.004550,0.002879,0.002247,0.003305,0.002416,0.001619,0.000981,0.000009,-0.000363,...,0.094537,0.098759,0.101977,0.095360,0.089466,0.095126,0.099496,0.093535,0.089035,0.090612
2,0.003531,0.002285,-0.000420,-0.003738,-0.006706,-0.003148,0.000733,0.000668,0.002162,-0.000946,...,0.085343,0.087155,0.084546,0.082166,0.081972,0.081413,0.081936,0.083011,0.082334,0.081487
3,-0.001772,-0.001311,0.000388,0.000408,-0.000355,0.000998,0.001109,-0.003149,-0.008882,-0.010483,...,0.088112,0.087322,0.083738,0.082701,0.084490,0.082785,0.084084,0.085761,0.083275,0.081404
4,0.000087,-0.000272,0.001022,0.003126,0.002284,0.000885,0.001933,0.002270,0.002247,0.002175,...,0.088028,0.090312,0.088713,0.086957,0.086522,0.081640,0.079652,0.081329,0.085397,0.088816


In [5]:
Body_Acc_X_test = pd.read_csv("/content/body_acc_x_test.txt", sep='\s+', header=None)
Body_Acc_y_test = pd.read_csv("/content/body_acc_y_test.txt", sep='\s+', header=None)
Body_Acc_z_test = pd.read_csv("/content/body_acc_z_test.txt", sep='\s+', header=None)
Body_Gyro_X_test = pd.read_csv("/content/body_gyro_x_test.txt", sep='\s+', header=None)
Body_Gyro_y_test = pd.read_csv("/content/body_gyro_y_test.txt", sep='\s+', header=None)
Body_Gyro_z_test = pd.read_csv("/content/body_gyro_z_test.txt", sep='\s+', header=None)
Total_Acc_X_test = pd.read_csv("/content/total_acc_x_test.txt", sep='\s+', header=None)
Total_Acc_y_test = pd.read_csv("/content/total_acc_y_test.txt", sep='\s+', header=None)
Total_Acc_z_test = pd.read_csv("/content/total_acc_z_test.txt", sep='\s+', header=None)


test_data = pd.concat([Body_Acc_X_test, Body_Acc_y_test, Body_Acc_z_test, Body_Gyro_X_test, Body_Gyro_y_test, Body_Gyro_z_test, Total_Acc_X_test, Total_Acc_y_test, Total_Acc_z_test], axis=1)

test_data.to_csv("test_data.csv", index=False)

In [6]:
test_data.shape

(2947, 1152)

In [7]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.011653,0.013109,0.011269,0.027831,0.002318,-0.018965,-0.061920,-0.094248,-0.079534,-0.070084,...,0.155110,0.153346,0.149289,0.147401,0.146905,0.145261,0.143904,0.144395,0.144703,0.145494
1,0.009280,0.004930,0.003954,0.009214,0.016156,0.014079,0.013105,0.021247,0.026823,0.031054,...,0.133731,0.132851,0.132054,0.132224,0.139767,0.147877,0.153025,0.152788,0.139843,0.121314
2,0.005732,0.007066,0.005110,0.002434,0.002024,0.001382,0.001930,0.004875,0.003776,0.002400,...,0.131392,0.132404,0.134938,0.137589,0.134431,0.133174,0.133712,0.132678,0.132694,0.132117
3,0.000452,0.000604,-0.002485,-0.004562,-0.006002,-0.006382,-0.005276,-0.001489,0.005153,0.004951,...,0.126040,0.124815,0.125353,0.126463,0.121757,0.117611,0.118865,0.116060,0.110997,0.111912
4,-0.004362,-0.002765,-0.004905,-0.004682,-0.000267,0.004135,0.005626,0.000933,-0.001542,0.003020,...,0.129091,0.125772,0.126042,0.121086,0.118511,0.125458,0.129365,0.125895,0.122510,0.122760


In [8]:
timesteps = 128
num_features = 9
X_train = train_data.values.reshape(-1, timesteps, num_features)

print("New X_train shape:", X_train.shape)


New X_train shape: (7352, 128, 9)


In [9]:
timesteps = 128
num_features = 9
X_test = test_data.values.reshape(-1, timesteps, num_features)

print("New X_train shape:", X_train.shape)

New X_train shape: (7352, 128, 9)


In [10]:
y = pd.read_csv("y_train.txt", sep='\s+', header=None)

In [11]:
ytest = pd.read_csv("y_test.txt", sep='\s+', header=None)

In [12]:
y_train = to_categorical(y.values.ravel())

In [13]:
ytest = to_categorical(ytest.values.ravel())

In [14]:
print(y_train)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [15]:
print(ytest)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [16]:
def evaluate_model(model, X_val, y_val):
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_val, axis=1)

    acc = accuracy_score(y_true_classes, y_pred_classes)
    prec = precision_score(y_true_classes, y_pred_classes, average='weighted')
    rec = recall_score(y_true_classes, y_pred_classes, average='weighted')
    f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    return acc, prec, rec, f1

In [ ]:
def build_cnn_model():
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(timesteps, num_features)),
        Dropout(0.5),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Train CNN
cnn_model = build_cnn_model()
cnn_model.summary()
cnn_model.fit(X_train, y_train, validation_data=(X_test, ytest), epochs=20, batch_size=32)

print("Evaluating CNN Model...")
evaluate_model(cnn_model, X_test, ytest)

In [18]:
# 2. MLP Model
def build_mlp_model():
    model = Sequential([
        Flatten(input_shape=(timesteps, num_features)),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])
    return model

# Train MLP
mlp_model = build_mlp_model()
mlp_model.summary()
mlp_model.fit(X_train, y_train, validation_data=(X_test, ytest), epochs=20, batch_size=32)

print("Evaluating MLP Model...")
evaluate_model(mlp_model, X_test, ytest)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)                  │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         590,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │           1,799 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 723,463 (2.76 MB)

 Trainable params: 723,463 (2.76 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.6967 - loss: 0.7965 - precision: 0.8223 - recall: 0.5749 - val_accuracy: 0.8459 - val_loss: 0.4830 - val_precision: 0.8546 - val_recall: 0.8337
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.8909 - loss: 0.2806 - precision: 0.9010 - recall: 0.8814 - val_accuracy: 0.8721 - val_loss: 0.5363 - val_precision: 0.8772 - val_recall: 0.8649
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9158 - loss: 0.2128 - precision: 0.9212 - recall: 0.9102 - val_accuracy: 0.8850 - val_loss: 0.3667 - val_precision: 0.8892 - val_recall: 0.8819
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9225 - loss: 0.1966 - precision: 0.9270 - recall: 0.9201 - val_accuracy: 0.8884 - val_loss: 0.3453 - val_precision: 0.8928 - val_recall: 0.8850
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9340 - loss: 0.1703 - precision: 0.9369 - recall: 0.9313 - val_accuracy: 0.8819 - val_los

(0.9043094672548354,
 0.9052763407746208,
 0.9043094672548354,
 0.9039258083669588)

In [19]:
# 3. LSTM Model
def build_lstm_model():
    model = Sequential([
        LSTM(64, input_shape=(timesteps, num_features), return_sequences=False),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])
    return model

# Train LSTM
lstm_model = build_lstm_model()
lstm_model.summary()
lstm_model.fit(X_train, y_train, validation_data=(X_test, ytest), epochs=20, batch_size=32)

print("Evaluating LSTM Model...")
evaluate_model(lstm_model, X_test, ytest)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 64)                  │          18,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 28,167 (110.03 KB)

 Trainable params: 28,167 (110.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 21s 80ms/step - accuracy: 0.3984 - loss: 1.3925 - precision: 0.7690 - recall: 0.1936 - val_accuracy: 0.5358 - val_loss: 1.1629 - val_precision: 0.8234 - val_recall: 0.3275
Epoch 2/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - accuracy: 0.5857 - loss: 0.9538 - precision: 0.8469 - recall: 0.3826 - val_accuracy: 0.5507 - val_loss: 1.0079 - val_precision: 0.7922 - val_recall: 0.3583
Epoch 3/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 22s 78ms/step - accuracy: 0.6153 - loss: 0.8678 - precision: 0.7839 - recall: 0.4274 - val_accuracy: 0.5948 - val_loss: 0.8950 - val_precision: 0.8988 - val_recall: 0.3376
Epoch 4/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 17s 75ms/step - accuracy: 0.6537 - loss: 0.7813 - precision: 0.7881 - recall: 0.4792 - val_accuracy: 0.5931 - val_loss: 0.8549 - val_precision: 0.6760 - val_recall: 0.4737
Epoch 5/20
230/230 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.7237 - loss: 0.6391 - precision: 0.7775 - recall: 0.6453 - val_accuracy: 0.7594 - va

(0.8109942314217848,
 0.8153395489652263,
 0.8109942314217848,
 0.8106492557279527)

In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, Flatten, Dense, LSTM, Input
from tensorflow.keras.utils import to_categorical

k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)


def initialize_metrics():
    return {"accuracy": [], "precision": [], "recall": [], "f1_score": []}


def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def build_mlp_model(input_shape, num_classes):
    model = Sequential([
        Flatten(input_shape=input_shape),  # Flatten the input
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def build_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def k_fold_evaluation(model_builder, X, y, model_name):
    metrics = initialize_metrics()
    fold = 1

    for train_index, val_index in kf.split(X):
        print(f"Fold {fold}/{k} - {model_name}")

        # Split data
        X_train_fold, X_val_fold = X[train_index], X[val_index]
        y_train_fold, y_val_fold = y[train_index], y[val_index]

        # Build and train model
        model = model_builder((X.shape[1], X.shape[2]), y.shape[1])
        model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=32, verbose=0)

        # Evaluate the model
        y_pred = model.predict(X_val_fold)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_val_fold, axis=1)

        # Calculate metrics
        acc = accuracy_score(y_true_classes, y_pred_classes)
        prec = precision_score(y_true_classes, y_pred_classes, average='weighted')
        rec = recall_score(y_true_classes, y_pred_classes, average='weighted')
        f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

        # Store results
        metrics["accuracy"].append(acc)
        metrics["precision"].append(prec)
        metrics["recall"].append(rec)
        metrics["f1_score"].append(f1)

        print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
        fold += 1

    # Print mean results
    print(f"\n{model_name} K-Fold Cross-Validation Results:")
    print(f"Mean Accuracy: {np.mean(metrics['accuracy']):.4f}")
    print(f"Mean Precision: {np.mean(metrics['precision']):.4f}")
    print(f"Mean Recall: {np.mean(metrics['recall']):.4f}")
    print(f"Mean F1 Score: {np.mean(metrics['f1_score']):.4f}\n")
    return metrics

# Perform K-Fold Cross-Validation for all models
print("Evaluating 1D-CNN...")
cnn_metrics = k_fold_evaluation(build_cnn_model, X_train, y_train, "1D-CNN")

print("Evaluating MLP...")
mlp_metrics = k_fold_evaluation(build_mlp_model, X_train, y_train, "MLP")

print("Evaluating LSTM...")
lstm_metrics = k_fold_evaluation(build_lstm_model, X_train, y_train, "LSTM")


Evaluating 1D-CNN...
Fold 1/5 - 1D-CNN


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step
Accuracy: 0.9748, Precision: 0.9750, Recall: 0.9748, F1: 0.9748
Fold 2/5 - 1D-CNN


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
Accuracy: 0.9755, Precision: 0.9757, Recall: 0.9755, F1: 0.9755
Fold 3/5 - 1D-CNN


...

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Accuracy: 0.9442, Precision: 0.9445, Recall: 0.9442, F1: 0.9441
Fold 4/5 - MLP


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Accuracy: 0.9510, Precision: 0.9519, Recall: 0.9510, F1: 0.9512
Fold 5/5 - MLP


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Accuracy: 0.9497, Precision: 0.9500, Recall: 0.9497, F1: 0.9497

MLP K-Fold Cross-Validation Results:
Mean Accuracy: 0.9498
Mean Precision: 0.9502
Mean Recall: 0.9498
Mean F1 Score: 0.9498

Evaluating LSTM...
Fold 1/5 - LSTM
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Accuracy: 0.9225, Precision: 0.9229, Recall: 0.9225, F1: 0.9226
Fold 2/5 - LSTM
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Accuracy: 0.9014, Precision: 0.9051, Recall: 0.9014, F1: 0.9013
Fold 3/5 - LSTM
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step
Accuracy: 0.9265, Precision: 0.9294, Recall: 0.9265, F1: 0.9268
Fold 4/5 - LSTM
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
Accuracy: 0.9041, Precision: 0.9088, Recall: 0.9041, F1: 0.9038
Fold 5/5 - LSTM
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Accuracy: 0.8905, Precision: 0.8983, Recall: 0.8905, F1: 0.8884

LSTM K-Fold Cross-Validation Results:
Mean Accuracy: 0.9090
Mean Precision: 0.9129
Mean Recall: 0.9090
Mean F1 Score: 0.9086



In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dropout, Flatten, Dense, LSTM, Input
from tensorflow.keras.utils import to_categorical
import pandas as pd

group_labels = np.loadtxt('subject_train.txt')

# Placeholder for scores
def initialize_metrics():
    return {"accuracy": [], "precision": [], "recall": [], "f1_score": []}


def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_mlp_model(input_shape, num_classes):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def build_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, return_sequences=False),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def loso_evaluation(model_builder, X, y, subject_ids, model_name):
    unique_subjects = np.unique(subject_ids)
    metrics = initialize_metrics()

    print(f"\nStarting LOSO-CV for {model_name}...")

    for subject in unique_subjects:
        print(f"Leaving out Subject {subject}...")


        train_indices = subject_ids != subject
        test_indices = subject_ids == subject

        X_train_fold, X_test_fold = X[train_indices], X[test_indices]
        y_train_fold, y_test_fold = y[train_indices], y[test_indices]

        model = model_builder((X.shape[1], X.shape[2]), y.shape[1])
        model.fit(X_train_fold, y_train_fold, epochs=20, batch_size=32, verbose=0)

        y_pred = model.predict(X_test_fold)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_test_fold, axis=1)


        acc = accuracy_score(y_true_classes, y_pred_classes)
        prec = precision_score(y_true_classes, y_pred_classes, average='weighted')
        rec = recall_score(y_true_classes, y_pred_classes, average='weighted')
        f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')


        metrics["accuracy"].append(acc)
        metrics["precision"].append(prec)
        metrics["recall"].append(rec)
        metrics["f1_score"].append(f1)

        print(f"Subject {subject} - Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")

    print(f"\n{model_name} LOSO-CV Results:")
    print(f"Mean Accuracy: {np.mean(metrics['accuracy']):.4f}")
    print(f"Mean Precision: {np.mean(metrics['precision']):.4f}")
    print(f"Mean Recall: {np.mean(metrics['recall']):.4f}")
    print(f"Mean F1 Score: {np.mean(metrics['f1_score']):.4f}\n")
    return metrics

print("Evaluating 1D-CNN with LOSO-CV...")
cnn_metrics_loso = loso_evaluation(build_cnn_model, X_train, y_train, group_labels, "1D-CNN")

print("Evaluating MLP with LOSO-CV...")
mlp_metrics_loso = loso_evaluation(build_mlp_model, X_train, y_train, group_labels, "MLP")

print("Evaluating LSTM with LOSO-CV...")
lstm_metrics_loso = loso_evaluation(build_lstm_model, X_train, y_train, group_labels, "LSTM")


Evaluating 1D-CNN with LOSO-CV...

Starting LOSO-CV for 1D-CNN...
Leaving out Subject 1.0...
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Subject 1.0 - Accuracy: 0.9481, Precision: 0.9565, Recall: 0.9481, F1: 0.9478
Leaving out Subject 3.0...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Subject 3.0 - Accuracy: 0.9384, Precision: 0.9456, Recall: 0.9384, F1: 0.9395
Leaving out Subject 5.0...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Subject 5.0 - Accuracy: 0.9139, Precision: 0.9361, Recall: 0.9139, F1: 0.9120
Leaving out Subject 6.0...


...

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Subject 27.0 - Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Leaving out Subject 28.0...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Subject 28.0 - Accuracy: 0.7644, Precision: 0.7932, Recall: 0.7644, F1: 0.7533
Leaving out Subject 29.0...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 